In [46]:
# Initial imports
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path

In [47]:
# call the weather file
file_path = "../Resources/NYC_weather_data_cleaned.csv"
weather_df = pd.read_csv(file_path, index_col=0 )
print(weather_df.shape)
weather_df.head(5)

(24336, 8)


,DATES,TIME,TEMP,VISIBILITY,HUMIDITY,WIND_SPEED,WEATHER_MAIN,HOUR
0,2020-01-01,00:00:00,44.19,10000.0,71,10.29,Clouds,0
1,2020-01-01,01:00:00,44.15,10000.0,71,10.29,Clouds,1
2,2020-01-01,02:00:00,42.82,10000.0,70,17.22,Rain,2
3,2020-01-01,03:00:00,42.58,10000.0,68,16.11,Rain,3
4,2020-01-01,04:00:00,42.31,10000.0,64,21.92,Clouds,4


In [48]:
# change TIME type from datetime to string inorder to be able to strip hours
weather_df["TIME"]=weather_df["TIME"].astype(str)

In [49]:
# make a column HOUR with hours from stripped
weather_df["HOUR"]= weather_df['TIME'].str[:2]

In [50]:
# check the new column
weather_df["HOUR"]

0        00
1        01
2        02
3        03
4        04
         ..
25696    19
25697    20
25698    21
25699    22
25700    23
Name: HOUR, Length: 24336, dtype: object

In [51]:
# dropping duplicates from the dataframe based on DATES and TIME 
weather_df1=weather_df.drop_duplicates(subset= ["DATES", "TIME"])

In [52]:
# change DATES type from string to Datetime
weather_df1["DATES"]= pd.to_datetime(weather_df1["DATES"])

In [53]:
# call the crash file 
file_path = "../Resources/NYC_Crash_Cyclist_Cleaned.csv"
crash_df = pd.read_csv(file_path, index_col=0 )
print(crash_df.shape)
crash_df.head(5)

(9193, 21)


,index,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,...,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,STREET
COLLISION_ID,,,,,,,,,,,,,,,,,,,,,
4271563,13297,1/2/20,13:00:00,MANHATTAN,10003,40.714165,-74.006320,"(40.714165, -74.00632)",1,0,...,0,1,0,0,0,Driver Inattention/Distraction,Unspecified,Sedan,Bike,CHAMBERS STREET
4268322,13344,1/2/20,16:00:00,MANHATTAN,10012,40.725643,-73.992070,"(40.725643, -73.99207)",1,0,...,0,1,0,0,0,Turning Improperly,Unspecified,Sedan,Bike,BOWERY
4268207,13347,1/2/20,17:00:00,BROOKLYN,11221,40.693874,-73.917770,"(40.693874, -73.91777)",1,0,...,0,1,0,0,0,Traffic Control Disregarded,Unspecified,Station Wagon/Sport Utility Vehicle,Bike,CENTRAL AVENUE
4268408,13304,1/3/20,8:00:00,BRONX,10456,40.820747,-73.906006,"(40.820747, -73.906006)",1,0,...,0,1,0,0,0,Driver Inattention/Distraction,Unspecified,E-Bike,Taxi,JACKSON AVENUE
4269187,13349,1/3/20,11:00:00,BROOKLYN,11205,40.691017,-73.954475,"(40.691017, -73.954475)",1,0,...,0,1,0,0,0,View Obstructed/Limited,Unsafe Speed,Sedan,E-Bike,DE KALB AVENUE


In [54]:
# set index column as index for the dataframe
crash_df = crash_df.reset_index().set_index('index')

In [55]:
# rename column names to match with weather data
crash_df.rename(columns = {'CRASH DATE':'DATES', 'CRASH TIME': 'TIME'}, inplace =True )  

In [56]:
# change DATES tybe into datatime
crash_df["DATES"]= pd.to_datetime(crash_df["DATES"])

In [57]:
# add hour column from stripped from TIME columns first two letter
crash_df["HOUR"]= crash_df['TIME'].str[:2]

In [58]:
crash_df["HOUR"]= crash_df["HOUR"].str.strip()

In [59]:
crash_df["HOUR"]

index
13297    13
13344    16
13347    17
13304    8:
13349    11
         ..
13966    1:
13971    6:
13969    11
13972    12
13974    20
Name: HOUR, Length: 9193, dtype: object

In [60]:
# merge weather and crash data based on DATES and HOUR columns
new = pd.merge(crash_df, weather_df1, how= "left", on=(['DATES', "HOUR"]))

In [61]:
print(new.shape)

(9193, 28)


In [62]:
# drop over lapping columns
new.drop(["TIME_y", "HOUR"], axis=1, inplace=True)

In [63]:
# keep one of the TIME column and change name
new.rename(columns={"TIME_x":"TIME"}, inplace=True)

In [64]:
# Drop location column as we have latitude and longitude column
new.drop(["LOCATION"], axis=1, inplace=True)

In [65]:
# accidently found that VEHICLE TYPE CODE 1 has upper and lower case letter, so unifor all as lower case
new["VEHICLE TYPE CODE 1"]=new["VEHICLE TYPE CODE 1"].str.lower()

In [66]:
new.to_csv("Crash_weather_merged.csv")